# **2199. Finding the Topic of Each Post**

**Table: Keywords**
``` sql
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| topic_id    | int     |
| word        | varchar |
+-------------+---------+
```
(topic_id, word) is the primary key (combination of columns with unique values) for this table.

Each row of this table contains the id of a topic and a word that is used to express this topic.

There may be more than one word to express the same topic and one word may be used to express multiple topics.
 

**Table: Posts**
``` sql
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| post_id     | int     |
| content     | varchar |
+-------------+---------+
```
post_id is the primary key (column with unique values) for this table.

Each row of this table contains the ID of a post and its content.

Content will consist only of English letters and spaces.
 

Leetcode has collected some posts from its social media website and is interested in finding the topics of each post. Each topic can be expressed by one or more keywords. If a keyword of a certain topic exists in the content of a post (case insensitive) then the post has this topic.

Write a solution to find the topics of each post according to the following rules:

If the post does not have keywords from any topic, its topic should be "Ambiguous!".

If the post has at least one keyword of any topic, its topic should be a string of the IDs of its topics sorted in ascending order and separated by commas ','. The string should not contain duplicate IDs.

Return the result table in any order.

The result format is in the following example.

 

**Example 1:**

**Input:** 
**Keywords table:**
``` sql
+----------+----------+
| topic_id | word     |
+----------+----------+
| 1        | handball |
| 1        | football |
| 3        | WAR      |
| 2        | Vaccine  |
+----------+----------+
```
**Posts table:**
``` sql
+---------+------------------------------------------------------------------------+
| post_id | content                                                                |
+---------+------------------------------------------------------------------------+
| 1       | We call it soccer They call it football hahaha                         |
| 2       | Americans prefer basketball while Europeans love handball and football |
| 3       | stop the war and play handball                                         |
| 4       | warning I planted some flowers this morning and then got vaccinated    |
+---------+------------------------------------------------------------------------+
```
**Output:**
``` sql 
+---------+------------+
| post_id | topic      |
+---------+------------+
| 1       | 1          |
| 2       | 1          |
| 3       | 1,3        |
| 4       | Ambiguous! |
+---------+------------+
```
**Explanation:** 

1: "We call it soccer They call it football hahaha"

"football" expresses topic 1. There is no other word that expresses any other topic.

2: "Americans prefer basketball while Europeans love handball and football"

"handball" expresses topic 1. "football" expresses topic 1. 

There is no other word that expresses any other topic.

3: "stop the war and play handball"

"war" expresses topic 3. "handball" expresses topic 1.

There is no other word that expresses any other topic.

4: "warning I planted some flowers this morning and then got vaccinated"

There is no word in this sentence that expresses any topic. Note that "warning" is different from "war" although they have a common prefix. 

This post is ambiguous.

Note that it is okay to have one word that expresses more than one topic.

**Solution-1:**

``` sql
with posts_with_words as(SELECT post_id, word
FROM Posts, 
JSON_TABLE(
    CONCAT('["', REPLACE(content, ' ', '","'), '"]'),
    '$[*]' COLUMNS (word VARCHAR(255) PATH '$')
) AS words)

,cte2 as(
select distinct post_id, 
case when a.word = b.word then topic_id else null end as topic
from posts_with_words a
INNER JOIN keywords b
on a.word = b.word 
HAVING topic is not null)

select a.post_id, 
COALESCE(GROUP_CONCAT(topic order by topic ASC SEPARATOR ','), 'Ambiguous!') AS topic
from  posts a
LEFT JOIN cte2 b
ON a.post_id = b.post_id
GROUP BY a.post_id 
ORDER BY a.post_id  ASC


**Solution-2:**

``` sql
SELECT 
    P.post_id, 
    IFNULL(GROUP_CONCAT(DISTINCT K.topic_id ORDER BY K.topic_id), 'Ambiguous!') AS topic
FROM Posts AS P
LEFT JOIN Keywords AS K
ON CONCAT(' ', LOWER(P.content), ' ') LIKE CONCAT('% ', LOWER(K.word), ' %')
GROUP BY P.post_id

**Solution-3:**

``` sql
-- Write your PostgreSQL query statement below
with words as (
    Select
    post_id,
    unnest(string_to_array(lower(content),' ')) as content
    from
    Posts
),
valid as (
    Select distinct
    post_id, 
    topic_id
    from
    words w 
    left join keywords kw on w.content = lower(kw.word)
    order by post_id, topic_id
),
combine as 
(
    Select 
    post_id,
    string_agg(topic_id::text,',') as topic
    from 
    valid
    group by post_id
)
select
post_id,
case when topic is null then 'Ambiguous!'
else topic end as topic
from
combine
order by post_id

**Solution-4:**

``` sql
-- Write your PostgreSQL query statement below
with data_cte as (
select distinct p.post_id, k.topic_id
from Posts p
left join Keywords k on concat(' ', p.content, ' ') ilike concat('% ', k.word, ' %') 
)
select post_id, coalesce(string_agg(topic_id::text, ',' order by topic_id), 'Ambiguous!') as topic
from data_cte
group by post_id;

**Solution-5:**

``` sql
# Write your MySQL query statement below
WITH cte AS (
  SELECT post_id, topic_id AS topic
  FROM Posts p
  LEFT JOIN Keywords k
  ON LOCATE(CONCAT(' ', word, ' '), CONCAT(' ', content, ' ')) > 0
)
SELECT post_id, GROUP_CONCAT(DISTINCT IFNULL(topic, "Ambiguous!") ORDER BY topic) AS topic
FROM cte
GROUP BY post_id

**Solution-6:**

``` sql
with agg as (
    select p.post_id,
           k.topic_id
      from Posts p left join
           Keywords k on p.content REGEXP concat('(^| )', k.word, '($| )')
  order by p.post_id,
           k.topic_id)

   select post_id,
          coalesce(group_concat(distinct topic_id), 'Ambiguous!') topic
     from agg 
 group by post_id 